# Section 5: Using aggregation pipelines

<div class="alert alert-block alert-info">
   
## Jupyter Notebook basics

- **Code cells:** Cells shaded grey are code cells. As you work through the lab, run all code cells in order.
- **Running code:** To run code, press Shift + Enter or click the 'Run' button on the menu bar. Where there is code already in a cell, run it as written. Where a code cell contains the comment `#Write your code here`, write code to complete the task & then run it. If needed, consult the hints & answer to enter and run the correct entry for a task before moving on to the next task. Not every command will result in visible output.
- **Markdown cells:** The non-code cells are written in the Markdown markup language. Double-clicking a Markdown cell will cause it to appear in raw Markdown format. To render as text again, run the cell just like running a code cell: press Shift + Enter or click the 'Run' button on the menu bar.  
- **Restarting kernel:** If the notebook becomes unresponsive, or if either the notebook or your code displays unexpected behavior, reset the notebook by choosing "Kernel -> Restart & Clear Output" from the menu bar. This will clear all memory objects in the notebook, stop any code running, and reset the notebook to its initial state. 
- **Session timeout:** Sessions will automatically shut down after about 10 minutes of inactivity. (If you leave a lab window open in the foreground, this will generally be counted as “activity”.) See Binder docs: [How long will my Binder session last?](https://mybinder.readthedocs.io/en/latest/about/about.html?highlight=session%20last#how-long-will-my-binder-session-last)
- **File navigation:** To navigate the other files in this lab, click on the folder icon (File Browser) at the top of the left sidebar and choose the `Contents.ipynb` file (or access the Contents file directly [here](../Contents.ipynb))

</div>

## Introduction

In this section you'll use aggregation pipelines to do more complex data processing and work with documents across multiple collections in a database. Specifically, you'll:

- Create aggregations using the `$match`, `$group`, `$sort`, `$limit`, `$unwind`, and `$project` stages  
- Create aggregations using the `$sum` and `$avg` aggregation operators
- Build an aggregation pipeline that joins data across two collections in the same database using the `$lookup` operator

The four tasks in this section are written with varying levels of scaffolding support for creating the aggregation pipelines. While the first two tasks provide support on identifying and coding the stages of an aggregation pipeline, by the end of the of the section you'll be writing aggregation pipelines on your own. 

## Setup 

Before starting on the tasks below, run the following cells. 

This sets up a new MongoDB client, connects it to the MongoDB server instance and sets up to query the `sample_mflix` database.  

In [ ]:
from pymongo import MongoClient
client = MongoClient()
db = client.sample_mflix

Run the cell below, which imports the Python `pprint` module and method. You'll use `pprint` to print output in a more readable format. 

In [ ]:
# Import the pprint method from the native Python pprint library
from pprint import pprint

## Tasks

### 1. Create an aggregation with `$match`, `$sort`, and `$limit` stages  
Using an aggregation pipeline, return the document for the movie that won the most awards in 2014.

In [ ]:
stage_match_year = { 
    "$match": {"year": 2014}
}

In [ ]:
stage_sort_awards_desc = {
    "$sort" : {"awards.wins": -1}
}

In [ ]:
stage_limit_1 = {
    "$limit" : 1
}

In [ ]:
pipeline = [
    stage_match_year,
    stage_sort_awards_desc,
    stage_limit_1,
]

In [ ]:
results = db.movies.aggregate(pipeline)

for movie in results:
    pprint(movie)

### 2. Create an aggregation with `$match`, `$group`, and `$sort` stages. Use the `$sum` operator. 
How many PG-13 movies does the database contain for each year? Order the results by the number of PG-13 movies, from most to least. 

In [ ]:
stage_match_rating = { 
    "$match": {"rated": "PG-13"}
}

In [ ]:
stage_group_by_year = { 
    "$group": {
        "_id": "$year",
        "movie_count": {"$sum" : 1},
    }
}

In [ ]:
stage_sort_by_rating_desc = {
    "$sort": {"movie_count": -1}
}

In [ ]:
pipeline = [
    stage_match_rating,
    stage_group_by_year,
    stage_sort_by_rating_desc,
]

In [ ]:
results = db.movies.aggregate(pipeline)

for movie in results:
    pprint(movie)

### 3. Create an aggregation with `$unwind`, `$group`,  `$sort` and `$limit` stages. Use the `$avg` operator.

Find the director who has the highest average IMDB rating over all of their movies in the `movies` database. 

In [ ]:
stage_unwind_directors = { 
    "$unwind": "$directors"
}

In [ ]:
stage_group_by_avg_imdb_rating = { 
    "$group": {
        "_id": "$directors",
        "avg_imdb": {"$avg" : "$imdb.rating"},
    }
}

In [ ]:
stage_sort_by_imdb_rating_desc = {
    "$sort": {"avg_imdb": -1}
}

In [ ]:
stage_limit_1 = {
    "$limit" : 1
}

In [ ]:
pipeline = [
    stage_unwind_directors,
    stage_group_by_avg_imdb_rating,
    stage_sort_by_imdb_rating_desc,
    stage_limit_1
]

In [ ]:
results = db.movies.aggregate(pipeline)

for movie in results:
    pprint(movie)

### 4. Create an aggregation with `$match`, `$lookup`,  and `$project` stages.  

We're are interested in data on comments written about the 2005 film "Fantastic Four". Joining the `movies` and `comments` databases, create a single document that consists of only the `_id`, movie title, year, and the name of the commenter. 

In [ ]:
stage_match_movie = { 
    "$match": {"title": "Fantastic Four", "year": 2015}
}

In [ ]:
stage_lookup_comments = {
   "$lookup": {
         "from": "comments",
         "localField": "_id",
         "foreignField": "movie_id",
         "as": "related_comments",
   }
}

In [ ]:
stage_project = {
    "$project" : { "title": 1, "year": 1, "related_comments" :1 }
}

In [ ]:
pipeline = [
   stage_match_movie,
   stage_lookup_comments,
   stage_project
]

In [ ]:
results = db.movies.aggregate(pipeline)
for movie in results:
    pprint(movie)

## Section wrap-up

Congratulations! In this section you used aggregation pipelines to do more complex data processing and work with documents across multiple collections in a database. Specifically, you:

- Created aggregations using the `$match`, `$group`, `$sort`, `$limit`, `$unwind`, and `$project` stages  
- Created aggregations using the `$sum` and `$avg` aggregation operators
- Built an aggregation pipeline that joined data across two collections in the same database using the `$lookup` operator

To learn more about using aggregation pipelines in MongoDB, you may be interested in checking out the [Practical MongoDB Aggregations](https://www.practical-mongodb-aggregations.com/) free e-book.